In [1]:
import requests
from urllib import request
from bs4 import BeautifulSoup
import time
import pandas as pd
from requests.compat import urlparse, urljoin
from requests.exceptions import HTTPError
import time
import sqlite3
from requests import Session
from bs4 import BeautifulSoup
import re
import datetime
import pandas as pd
import datetime
from tqdm import tqdm

In [2]:
ACODE = {"호텔신라":"008770","두산퓨얼셀":"336260","한화솔루션":"009830",
            "에코프로":"086520","신성이엔지":"011930","유니슨":"018000",
            "카카오":"035720","네이버":"035420","엔씨소프트":"036570",
            "sds":"018260","삼성전자":"005930","SK하이닉스":"000660",
            "skc":"011790","한솔케미칼":"014680","DB하이텍":"000990",
            "롯데쇼핑":"023530","신세계":"004170","삼성물산":"028260",
            "동서":"026960","kb금융":"105560","한국금융지주":"071050","신한지주":"055550",
            "우리금융지주":"316140","하나금융지주":"086790",
         "넥슨지티":"041140","한빛소프트":"047080","코세스":"089890",
      "드래곤플라이":"030350","에이트원":"230980","엔텔스":"069410"}


In [3]:
HEADERS = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36'}
DATE_STOP = '2021-12-23'


def title_crawl(ls, ticker, result):
    global DATE_STOP

    for i in range(0,len(ls)-1):
        response = requests.get('https://gall.dcinside.com' + str(ls[i]),headers = HEADERS)
        soup = BeautifulSoup(response.content,'html.parser')
        items = soup.find_all("table",{'class':'gall_list'})
        contents = soup.find('tbody').find_all('tr')
      
        for j in contents:         
            if j.find('td',{'class':'gall_writer ub-writer'}).text=='운영자':
                pass
            else:
                # 날짜
                date_dict = j.find('td',{'class':'gall_date'}).attrs

                if date_dict['title'][:10] <= DATE_STOP: 
                    return -1
                else:
                    # 제목
                    title = j.find('a').text
                    # 추천수
                    recommend_tag = j.find('td', class_='gall_recommend')
                    recommend = recommend_tag.text

                    # 조회수
                    views_tag = j.find('td', class_='gall_count')
                    views = views_tag.text

                    put_data = {
                        'code': ACODE[ticker],
                        'date' : date_dict['title'],
                        'title' : title,
                        'view' : views,
                        'recommend' : recommend
                    }

                    result.append(put_data)
                    print(result)
    

In [4]:
def DC_crawling(ticker, page):
    result = []
    url = f"https://gall.dcinside.com/board/lists?id=neostock&s_type=search_subject_memo&s_keyword={ticker}"

    a = f'/board/lists?id=neostock&s_type=search_subject_memo&s_keyword={ticker}'
    response = requests.get(url,headers=HEADERS)
    soup = BeautifulSoup(response.text,'html.parser')
    items = soup.find('div',{'class':"bottom_paging_box"})

    #리스트 만들기 url
    url_list=[a]
    for i in items.find_all('a'):
        url_url = i['href']
        url_list.append(url_url)

    #  첫페이지
    title_crawl(url_list, ticker, result)
    
    k = 0
    while k < page :
        response = requests.get('https://gall.dcinside.com' + str(url_list[-1]),headers=HEADERS)
        soup = BeautifulSoup(response.content,'html.parser')
        items = soup.find('div',{'class':"bottom_paging_box"})
        url_list=[str(url_list[-1])]

        for i in items.find_all('a'):
            url_url = i['href']
            url_list.append(url_url)
        url_list.pop(1) 
        
        if title_crawl(url_list, ticker,result) == -1:
            break
        else:
            k += 1

    return result


In [39]:
for t in ACODE.keys():
    DC_crawling(t, 10)

[{'code': '035720', 'date': '2021-12-27 17:19:49', 'title': '20년동안 짝사랑 했던 여자애가 결혼한다.', 'view': '78', 'recommend': '2'}]
[{'code': '035720', 'date': '2021-12-27 17:19:49', 'title': '20년동안 짝사랑 했던 여자애가 결혼한다.', 'view': '78', 'recommend': '2'}, {'code': '035720', 'date': '2021-12-27 16:29:17', 'title': '권력밀착 카카오 탈세 고발', 'view': '41', 'recommend': '2'}]
[{'code': '035720', 'date': '2021-12-27 17:19:49', 'title': '20년동안 짝사랑 했던 여자애가 결혼한다.', 'view': '78', 'recommend': '2'}, {'code': '035720', 'date': '2021-12-27 16:29:17', 'title': '권력밀착 카카오 탈세 고발', 'view': '41', 'recommend': '2'}, {'code': '035720', 'date': '2021-12-27 15:00:48', 'title': '지금이 매수를 지켜볼 때', 'view': '43', 'recommend': '0'}]
[{'code': '035720', 'date': '2021-12-27 17:19:49', 'title': '20년동안 짝사랑 했던 여자애가 결혼한다.', 'view': '78', 'recommend': '2'}, {'code': '035720', 'date': '2021-12-27 16:29:17', 'title': '권력밀착 카카오 탈세 고발', 'view': '41', 'recommend': '2'}, {'code': '035720', 'date': '2021-12-27 15:00:48', 'title': '지금이 매수를 지켜볼 때', 'view

[{'code': '005930', 'date': '2021-12-27 10:55:08', 'title': '삼성전자사라', 'view': '84', 'recommend': '2'}]
[{'code': '005930', 'date': '2021-12-27 10:55:08', 'title': '삼성전자사라', 'view': '84', 'recommend': '2'}, {'code': '005930', 'date': '2021-12-27 10:50:53', 'title': '주식얘기나해라 ㅆㅂㄹㄷㅇ', 'view': '111', 'recommend': '9'}]
[{'code': '004170', 'date': '2021-12-27 11:38:39', 'title': '광주 홍어들 미쳤네 ㅋㅋㅋㅋㅋ', 'view': '73', 'recommend': '5'}]
[{'code': '028260', 'date': '2021-12-27 13:38:13', 'title': '이재명 주식종목 가져왔다 개념보내줘라', 'view': '77', 'recommend': '3'}]
[{'code': '105560', 'date': '2021-12-27 13:38:13', 'title': '이재명 주식종목 가져왔다 개념보내줘라', 'view': '77', 'recommend': '3'}]


In [5]:
import time
import random
import datetime
from kafka import KafkaProducer

bootstrap_servers = ['localhost:9092']  # kafka broker ip
topicName = 'DC'
producer = KafkaProducer(bootstrap_servers=bootstrap_servers)


for value in ACODE.keys():
    print(value)
    DC_con = DC_crawling(value, 100)
    

    for i in DC_con:
        producer.send(topicName, str(i).encode())      
        tim = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        metadata = producer.send(topicName, tim.encode())
        print(i," - ",metadata)
#producer.flush()

#        time.sleep(1)


호텔신라
두산퓨얼셀
한화솔루션
에코프로
[{'code': '086520', 'date': '2021-12-27 20:39:13', 'title': '에코프로에이치엔 ㄹㅇ 뒤진 주식이냐?????????????', 'view': '39', 'recommend': '1'}]
[{'code': '086520', 'date': '2021-12-27 20:39:13', 'title': '에코프로에이치엔 ㄹㅇ 뒤진 주식이냐?????????????', 'view': '39', 'recommend': '1'}, {'code': '086520', 'date': '2021-12-24 09:16:31', 'title': '와씨 에코프로hn은 진짜 반등도없냐??????????', 'view': '37', 'recommend': '0'}]
[{'code': '086520', 'date': '2021-12-27 20:39:13', 'title': '에코프로에이치엔 ㄹㅇ 뒤진 주식이냐?????????????', 'view': '39', 'recommend': '1'}, {'code': '086520', 'date': '2021-12-24 09:16:31', 'title': '와씨 에코프로hn은 진짜 반등도없냐??????????', 'view': '37', 'recommend': '0'}, {'code': '086520', 'date': '2021-12-24 08:46:41', 'title': '2차전지 관련주 어떤게 갠츈함?', 'view': '169', 'recommend': '0'}]
{'code': '086520', 'date': '2021-12-27 20:39:13', 'title': '에코프로에이치엔 ㄹㅇ 뒤진 주식이냐?????????????', 'view': '39', 'recommend': '1'}  -  <kafka.producer.future.FutureRecordMetadata object at 0x7f853127f5e0>
{'code': '086520', 'date':

[{'code': '035420', 'date': '2021-12-28 00:50:58', 'title': '흠 내가 네이버 카카오 갈 수 있을까?', 'view': '31', 'recommend': '0'}, {'code': '035420', 'date': '2021-12-27 19:33:11', 'title': '내가 신이라면 이딴생각은 왜 하는거임?', 'view': '53', 'recommend': '0'}, {'code': '035420', 'date': '2021-12-27 15:33:34', 'title': '네이버 세일 하는데 왜 안사냐?', 'view': '47', 'recommend': '0'}, {'code': '035420', 'date': '2021-12-27 10:22:49', 'title': '조선 리딩방 동호회 따까리 끄나풀들 영업질 잘하네 ㅋㅋㅋ 이제 대놓고', 'view': '22', 'recommend': '0'}, {'code': '035420', 'date': '2021-12-27 09:31:51', 'title': '네이버 사자마자 물림 ㅜ', 'view': '160', 'recommend': '0'}, {'code': '035420', 'date': '2021-12-27 09:10:04', 'title': '연초까지 대형주 위주로 투자해라', 'view': '164', 'recommend': '0'}, {'code': '035420', 'date': '2021-12-27 08:58:35', 'title': '난 현시점에선 삼전보단 네이버가 매력적이라고 본다', 'view': '207', 'recommend': '0'}, {'code': '035420', 'date': '2021-12-27 00:34:38', 'title': '대체 MZ세대 << 이거 왜 쓰는거임?', 'view': '68', 'recommend': '3'}, {'code': '035420', 'date': '2021-12-26 13:50:48', 'ti

[{'code': '036570', 'date': '2021-12-26 11:59:50', 'title': '주식 개장하면 엔씨소프트 사라(이유있음)', 'view': '69', 'recommend': '0'}]
[{'code': '036570', 'date': '2021-12-26 11:59:50', 'title': '주식 개장하면 엔씨소프트 사라(이유있음)', 'view': '69', 'recommend': '0'}, {'code': '036570', 'date': '2021-12-24 10:21:43', 'title': '엔씨소프트 풀매수해라', 'view': '78', 'recommend': '0'}]
[{'code': '036570', 'date': '2021-12-26 11:59:50', 'title': '주식 개장하면 엔씨소프트 사라(이유있음)', 'view': '69', 'recommend': '0'}, {'code': '036570', 'date': '2021-12-24 10:21:43', 'title': '엔씨소프트 풀매수해라', 'view': '78', 'recommend': '0'}, {'code': '036570', 'date': '2021-12-24 07:33:36', 'title': '한국을 주식에 비교하면 엔씨소프트 같음 ', 'view': '48', 'recommend': '0'}]
{'code': '036570', 'date': '2021-12-26 11:59:50', 'title': '주식 개장하면 엔씨소프트 사라(이유있음)', 'view': '69', 'recommend': '0'}  -  <kafka.producer.future.FutureRecordMetadata object at 0x7f85315a3880>
{'code': '036570', 'date': '2021-12-24 10:21:43', 'title': '엔씨소프트 풀매수해라', 'view': '78', 'recommend': '0'}  -  <kafka.pro

[{'code': '005930', 'date': '2021-12-27 23:00:02', 'title': '야 삼성전자 1월에 최고가 얼마였지?', 'view': '67', 'recommend': '0'}, {'code': '005930', 'date': '2021-12-27 10:55:08', 'title': '삼성전자사라', 'view': '93', 'recommend': '2'}, {'code': '005930', 'date': '2021-12-27 10:50:53', 'title': '주식얘기나해라 ㅆㅂㄹㄷㅇ', 'view': '118', 'recommend': '9'}, {'code': '005930', 'date': '2021-12-26 21:36:55', 'title': 'TSMC랑 삼성전자 같이 투자 하면 안됨? 5:5', 'view': '36', 'recommend': '0'}, {'code': '005930', 'date': '2021-12-26 21:29:23', 'title': '삼성전자 때문에 동생이랑 싸운 썰', 'view': '179', 'recommend': '0'}, {'code': '005930', 'date': '2021-12-26 16:37:05', 'title': '삼성전자 월요일에 가격 내려가나요?', 'view': '90', 'recommend': '1'}, {'code': '005930', 'date': '2021-12-26 15:30:43', 'title': '몇달전에 8만원 처음 갔는데 저번주 목요일 종가 7만 9900원 ㅋㅋㅋ', 'view': '43', 'recommend': '1'}, {'code': '005930', 'date': '2021-12-26 15:19:31', 'title': '전세계 주식은 삼성전자 1종목 밖에 없음.   삼성전자 아니면 주식이 아', 'view': '45', 'recommend': '1'}, {'code': '005930', 'date': '2021-12-26 15:04:56

[{'code': '000660', 'date': '2021-12-26 13:50:48', 'title': '한국기업 주식시장의 현실 ', 'view': '150', 'recommend': '2'}]
{'code': '000660', 'date': '2021-12-26 13:50:48', 'title': '한국기업 주식시장의 현실 ', 'view': '150', 'recommend': '2'}  -  <kafka.producer.future.FutureRecordMetadata object at 0x7f8531917bb0>
skc
한솔케미칼
DB하이텍
롯데쇼핑
신세계
[{'code': '004170', 'date': '2021-12-28 05:37:52', 'title': '실제후기) 이탈리아녀랑 결혼해라 진짜 ㅆㅂ 신세계다', 'view': '120', 'recommend': '3'}]
[{'code': '004170', 'date': '2021-12-28 05:37:52', 'title': '실제후기) 이탈리아녀랑 결혼해라 진짜 ㅆㅂ 신세계다', 'view': '120', 'recommend': '3'}, {'code': '004170', 'date': '2021-12-27 11:38:39', 'title': '광주 홍어들 미쳤네 ㅋㅋㅋㅋㅋ', 'view': '75', 'recommend': '5'}]
[{'code': '004170', 'date': '2021-12-28 05:37:52', 'title': '실제후기) 이탈리아녀랑 결혼해라 진짜 ㅆㅂ 신세계다', 'view': '120', 'recommend': '3'}, {'code': '004170', 'date': '2021-12-27 11:38:39', 'title': '광주 홍어들 미쳤네 ㅋㅋㅋㅋㅋ', 'view': '75', 'recommend': '5'}, {'code': '004170', 'date': '2021-12-26 21:29:55', 'title': '갑자기 연ﾠ애 존나 마려워서 급